In [1]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
import time
import random
from PIL import Image
import matplotlib.pyplot as plt

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


<h3>Dataset creation</h3>

In [3]:
data = np.load("data/images.npy")
data = torch.FloatTensor(np.expand_dims(data,axis=1))
data_permuts = torch.randperm(data.shape[0])
data = data[data_permuts,:]
labels = torch.FloatTensor(np.load("data/labels.npy"))

print(f"data: {data.shape}, labels: {labels.shape}")

data: torch.Size([18000, 1, 150, 150]), labels: torch.Size([18000, 2])


In [4]:
class ClockDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]
        return x, y
    
    def __len__(self):
        return len(self.data)

In [5]:
# Create main dataset
clock_dataset = ClockDataset(data, labels)
train_data, test_data, val_data = random_split(clock_dataset, [14000,3000,1000])

# Split dataset into train, test and validation sets
train_data = DataLoader(train_data, batch_size=32, shuffle=True)
test_data = DataLoader(test_data, batch_size=32, shuffle=True)
val_data = DataLoader(val_data, batch_size=32, shuffle=True)

In [6]:
for X, y in train_data:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([32, 1, 150, 150])
Shape of y: torch.Size([32, 2]) torch.float32


In [7]:
class NN_regression(nn.Module):
    """ Convolution model that returns one value as output.
    """
    def __init__(self, input_channels, h, w, n_outputs):
        super(NN_regression, self).__init__()

        self.input_layer = nn.Sequential(
            nn.Conv2d(input_channels, 32, kernel_size=5, stride=3),
            nn.BatchNorm2d(32)
        )
        self.hidden_layers = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=2),
            nn.ReLU(),
            nn.Flatten()
        )


        self.output_layer = nn.Sequential(
            nn.Linear(33856, 256),
            nn.ReLU(),
            nn.Linear(256, n_outputs)
            
        )


    def forward(self, x):
        x = self.input_layer(x)
        x = self.hidden_layers(x)
        x = self.output_layer(x)
        return x


In [8]:
model = NN_regression(input_channels=1,h=150,w=150, n_outputs=2).to(device)

<h3>Creating loss function</h3>

In [9]:
def common_sense_loss(prediction, target):
    """ Defines the common sense error in minutes.
    """

    # hour common sense error in minutes
    hour_start_idx = torch.minimum(prediction[0], target[0])
    hour_end_idx = torch.maximum(prediction[0], target[0])
    hour_dist_mins = (hour_end_idx - hour_start_idx)*60

    # minutes common sense error
    min_start_idx = torch.minimum(prediction[1], target[1])
    min_end_idx = torch.maximum(prediction[1], target[1])
    mins_dist_mins = (min_end_idx - min_start_idx)

    # return total common sense error in minutes
    return torch.mean(hour_dist_mins + mins_dist_mins)

a = torch.Tensor([[1,4,2,3],[14,28,36,1]])
b = torch.Tensor([[1,3,2,4],[15,29,36,1]])


In [10]:
class CommonSenseError(nn.Module):
    def __init__(self):
        super(CommonSenseError, self).__init__()

    def forward(self, prediction, target):
        # hour common sense error in minutes
        hour_start_idx = torch.minimum(prediction[0], target[0])
        hour_end_idx = torch.maximum(prediction[0], target[0])
        hour_dist_mins = (hour_end_idx - hour_start_idx)*60

        # minutes common sense error
        min_start_idx = torch.minimum(prediction[1], target[1])
        min_end_idx = torch.maximum(prediction[1], target[1])
        mins_dist_mins = (min_end_idx - min_start_idx)

        # return total common sense error in minutes
        return torch.mean(hour_dist_mins + mins_dist_mins)

In [11]:
loss = CommonSenseError()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [12]:
def train(dataloader, model, loss, optimizer):
    model.train()
    for X, y in dataloader:
        X, y = X.to(device), y.to(device)
        
        # Compute prediction error
        pred = model(X)
        loss = common_sense_loss(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Train loss: {loss:>7f}")

In [13]:
def test(dataloader, model, loss_fn):
    model.eval()
    num_batches = len(dataloader)
    test_loss = 0.0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            loss = loss_fn(pred, y)
            test_loss += loss
    test_loss /= num_batches
    print(f"Test avg loss: {test_loss:>8f} \n")

In [14]:
epochs = 50
start_time = time.time()
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_data, model, loss, optimizer)
    test(test_data, model, loss)
end_time = time.time()

Epoch 1
-------------------------------
Train loss: 401.575989
Test avg loss: 629.117920 

Epoch 2
-------------------------------
Train loss: 272.310608
Test avg loss: 576.533081 

Epoch 3
-------------------------------


In [ ]:
print(f"Done in {end_time - start_time}")

Done in 31.306610345840454


In [ ]:
one = 122.95
two = 124.95
tree = 84.58
four = 84.43